## Bibliotecas Necessárias

In [67]:
import matplotlib.pyplot as plt
import numpy as np

## Conversão ASCII para Binário

In [68]:
def texto_para_binario(mensagem):
    binario_completo = ""

    for caractere in mensagem:
        codigo_ascii = ord(caractere)
        binario = format(codigo_ascii, '08b')
        binario_completo += binario

    return binario_completo

## Codificação do Canal (NRZ Polar)

In [69]:
def nrz_polar(bits):
    sinal = []

    for bit in bits:
        if bit == '0':
            sinal.append(-1)
        else:
            sinal.append(+1)

    return sinal

## Modulação Digital

### Modulação BPSK

In [70]:
def modular_bpsk(sinal_nrz, frequencia_portadora=5, amostras_por_bit=100):
    sinal_modulado = []
    tempo = []

    for i, amplitude in enumerate(sinal_nrz):
        t = np.linspace(i, i + 1, amostras_por_bit, endpoint=False)
        portadora = amplitude * np.cos(2 * np.pi * frequencia_portadora * t)
        sinal_modulado.extend(portadora)
        tempo.extend(t)

    return np.array(sinal_modulado), np.array(tempo)

### Modulação QSPK

In [71]:
def modular_qpsk(sinal_nrz, frequencia_portadora=5, amostras_por_bit=100):
    sinal_modulado = []
    tempo = []

    for i in range(0, len(sinal_nrz), 2):
        bit1 = sinal_nrz[i]
        bit2 = sinal_nrz[i + 1]

        if bit1 == -1 and bit2 == -1:
            fase = 5 * np.pi / 4
        elif bit1 == -1 and bit2 == +1:
            fase = 3 * np.pi / 4
        elif bit1 == +1 and bit2 == -1:
            fase = 7 * np.pi / 4
        else:
            fase = np.pi / 4

        t = np.linspace(i/2, i/2 + 1, amostras_por_bit, endpoint=False)
        portadora = np.cos(2 * np.pi * frequencia_portadora * t + fase)

        sinal_modulado.extend(portadora)
        tempo.extend(t)

    return np.array(sinal_modulado), np.array(tempo)

## Ruido (AWGN)

In [72]:
def adicionar_awgn(sinal, snr_db):
    potencia_sinal = np.mean(sinal ** 2)
    snr_linear = 10 ** (snr_db / 10)
    potencia_ruido = potencia_sinal / snr_linear
    ruido = np.random.normal(0, np.sqrt(potencia_ruido), len(sinal))
    sinal_ruidoso = sinal + ruido

    return sinal_ruidoso, ruido

## Demodulação

### Demodulação BPSK

In [73]:
def demodular_bpsk(sinal_modulado, frequencia_portadora=5, amostras_por_bit=100):
    sinal_demodulado = []
    n_bits = len(sinal_modulado) // amostras_por_bit

    for i in range(n_bits):
        inicio = i * amostras_por_bit
        fim = inicio + amostras_por_bit
        sinal_bit = sinal_modulado[inicio:fim]

        t = np.linspace(i, i + 1, amostras_por_bit, endpoint=False)
        portadora = np.cos(2 * np.pi * frequencia_portadora * t)

        produto = sinal_bit * portadora
        valor_integrado = np.sum(produto)

        if valor_integrado > 0:
            sinal_demodulado.append(+1)
        else:
            sinal_demodulado.append(-1)

    return sinal_demodulado

### Demoludação QPSK

In [74]:
def demodular_qpsk(sinal_modulado, frequencia_portadora=5, amostras_por_bit=100):
    sinal_demodulado = []
    n_simbolos = len(sinal_modulado) // amostras_por_bit

    fases = {
        (5 * np.pi / 4): (-1, -1),
        (3 * np.pi / 4): (-1, +1),
        (7 * np.pi / 4): (+1, -1),
        (np.pi / 4): (+1, +1)
    }

    for i in range(n_simbolos):
        inicio = i * amostras_por_bit
        fim = inicio + amostras_por_bit
        sinal_simbolo = sinal_modulado[inicio:fim]

        t = np.linspace(i, i + 1, amostras_por_bit, endpoint=False)
        correlacoes = {}

        for fase, bits in fases.items():
            portadora = np.cos(2 * np.pi * frequencia_portadora * t + fase)
            correlacao = np.sum(sinal_simbolo * portadora)
            correlacoes[correlacao] = bits

        max_correlacao = max(correlacoes.keys())
        bit1, bit2 = correlacoes[max_correlacao]

        sinal_demodulado.append(bit1)
        sinal_demodulado.append(bit2)

    return sinal_demodulado

## Decodificação

In [75]:
def decodificar_nrz_polar(sinal_nrz):
    bits = ""
    for simbolo in sinal_nrz:
        if simbolo < 0:
            bits += '0'
        else:
            bits += '1'
    return bits

def binario_para_texto(bits):
    mensagem = ""
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if len(byte) == 8:
            mensagem += chr(int(byte, 2))
    return mensagem

## Calcular BER (Bit Error Rate)

In [76]:
def calcular_ber(bits_originais, bits_recebidos):
    tamanho_min = min(len(bits_originais), len(bits_recebidos))
    bits_orig = bits_originais[:tamanho_min]
    bits_recv = bits_recebidos[:tamanho_min]

    erros = sum(b1 != b2 for b1, b2 in zip(bits_orig, bits_recv))
    ber = erros / tamanho_min if tamanho_min > 0 else 0

    return ber, erros

## Plotagem

### Comportamento da BER em função da SNR

In [77]:
def plotar_ber_vs_snr_comparativo(mensagem, snr_range=range(-5, 16), amostras_por_bit=100):

    # Converter mensagem para binário
    bits_originais = texto_para_binario(mensagem)

    # Garantir número par de bits para QPSK
    if len(bits_originais) % 2 != 0:
        bits_originais += '0'

    # Codificar em NRZ-Polar
    sinal_nrz = nrz_polar(bits_originais)

    ber_bpsk = []
    ber_qpsk = []
    snr_values = list(snr_range)

    print(f"Mensagem: '{mensagem}'")
    print(f"Total de bits: {len(bits_originais)}\n")

    for snr_db in snr_values:
        # ============ BPSK ============
        # Modular
        sinal_bpsk, _ = modular_bpsk(sinal_nrz, amostras_por_bit=amostras_por_bit)

        # Adicionar ruído
        sinal_bpsk_ruidoso, _ = adicionar_awgn(sinal_bpsk, snr_db)

        # Demodular
        sinal_demod_bpsk = demodular_bpsk(sinal_bpsk_ruidoso, amostras_por_bit=amostras_por_bit)

        # Decodificar
        bits_recebidos_bpsk = decodificar_nrz_polar(sinal_demod_bpsk)

        # Calcular BER
        ber, _ = calcular_ber(bits_originais, bits_recebidos_bpsk)
        ber_bpsk.append(ber)

        # ============ QPSK ============
        # Modular
        sinal_qpsk, _ = modular_qpsk(sinal_nrz, amostras_por_bit=amostras_por_bit)

        # Adicionar ruído
        sinal_qpsk_ruidoso, _ = adicionar_awgn(sinal_qpsk, snr_db)

        # Demodular
        sinal_demod_qpsk = demodular_qpsk(sinal_qpsk_ruidoso, amostras_por_bit=amostras_por_bit)

        # Decodificar
        bits_recebidos_qpsk = decodificar_nrz_polar(sinal_demod_qpsk)

        # Calcular BER
        ber, _ = calcular_ber(bits_originais, bits_recebidos_qpsk)
        ber_qpsk.append(ber)

        print(f"SNR = {snr_db:3d} dB | BER BPSK = {ber_bpsk[-1]:.6f} | BER QPSK = {ber_qpsk[-1]:.6f}")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Subplot 1: BPSK
    ax1.semilogy(snr_values, ber_bpsk, 'b-o', linewidth=2, markersize=6)
    ax1.set_xlabel('SNR (dB)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Taxa de Erro de Bits (BER)', fontsize=12, fontweight='bold')
    ax1.set_title('BPSK - BER vs SNR', fontsize=14, fontweight='bold')
    ax1.grid(True, which='both', linestyle='--', alpha=0.6)
    ax1.set_xlim(min(snr_values), max(snr_values))
    ax1.set_ylim(1e-4, 1)

    # Subplot 2: QPSK
    ax2.semilogy(snr_values, ber_qpsk, 'r-s', linewidth=2, markersize=6)
    ax2.set_xlabel('SNR (dB)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Taxa de Erro de Bits (BER)', fontsize=12, fontweight='bold')
    ax2.set_title('QPSK - BER vs SNR', fontsize=14, fontweight='bold')
    ax2.grid(True, which='both', linestyle='--', alpha=0.6)
    ax2.set_xlim(min(snr_values), max(snr_values))
    ax2.set_ylim(1e-4, 1)

    plt.tight_layout()
    plt.savefig('ber_vs_snr_comparativo.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"\nGráfico comparativo salvo como 'ber_vs_snr_comparativo.png'")

    return snr_values, ber_bpsk, ber_qpsk

### Metrificação



In [78]:
def plotar_sinais_exemplo(mensagem, snr_range=range(-5, 16), amostras_por_bit=100):
    bits = texto_para_binario(mensagem)
    if len(bits) % 2 != 0:
        bits += '0'

    sinal_nrz = nrz_polar(bits)

    # Limitar número de bits para visualização
    n_bits_plot = min(16, len(bits))
    bits_plot = bits[:n_bits_plot]
    sinal_nrz_plot = sinal_nrz[:n_bits_plot]

    print(f"\nGerando gráficos de sinais para cada SNR")
    print(f"Mensagem: '{mensagem}'")
    print(f"Primeiros {n_bits_plot} bits: {bits_plot}\n")

    for snr_db in snr_range:
        # BPSK
        sinal_bpsk, tempo_bpsk = modular_bpsk(sinal_nrz_plot, amostras_por_bit=amostras_por_bit)
        sinal_bpsk_ruidoso, _ = adicionar_awgn(sinal_bpsk, snr_db)

        # QPSK
        sinal_qpsk, tempo_qpsk = modular_qpsk(sinal_nrz_plot, amostras_por_bit=amostras_por_bit)
        sinal_qpsk_ruidoso, _ = adicionar_awgn(sinal_qpsk, snr_db)

        # Plotar
        fig, axes = plt.subplots(3, 2, figsize=(16, 12))
        fig.suptitle(f'Sinais com SNR = {snr_db} dB', fontsize=16, fontweight='bold', y=0.995)

        # Coluna 1: BPSK
        # Sinal NRZ
        axes[0, 0].step(range(len(sinal_nrz_plot)), sinal_nrz_plot, 'b-', linewidth=2, where='post')
        axes[0, 0].set_title('Sinal NRZ-Polar', fontsize=12, fontweight='bold')
        axes[0, 0].set_ylabel('Amplitude', fontsize=10)
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].set_ylim(-1.5, 1.5)

        # BPSK sem ruído
        axes[1, 0].plot(tempo_bpsk, sinal_bpsk, 'b-', linewidth=1)
        axes[1, 0].set_title(f'Sinal BPSK Modulado', fontsize=12, fontweight='bold')
        axes[1, 0].set_ylabel('Amplitude', fontsize=10)
        axes[1, 0].grid(True, alpha=0.3)

        # BPSK com ruído
        axes[2, 0].plot(tempo_bpsk, sinal_bpsk_ruidoso, 'b-', linewidth=1, alpha=0.7)
        axes[2, 0].set_title(f'Sinal BPSK com Ruído', fontsize=12, fontweight='bold')
        axes[2, 0].set_xlabel('Tempo (s)', fontsize=10)
        axes[2, 0].set_ylabel('Amplitude', fontsize=10)
        axes[2, 0].grid(True, alpha=0.3)

        # Coluna 2: QPSK
        # Sinal NRZ (repetido para simetria)
        axes[0, 1].step(range(len(sinal_nrz_plot)), sinal_nrz_plot, 'r-', linewidth=2, where='post')
        axes[0, 1].set_title('Sinal NRZ-Polar', fontsize=12, fontweight='bold')
        axes[0, 1].set_ylabel('Amplitude', fontsize=10)
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].set_ylim(-1.5, 1.5)

        # QPSK sem ruído
        axes[1, 1].plot(tempo_qpsk, sinal_qpsk, 'r-', linewidth=1)
        axes[1, 1].set_title(f'Sinal QPSK Modulado', fontsize=12, fontweight='bold')
        axes[1, 1].set_ylabel('Amplitude', fontsize=10)
        axes[1, 1].grid(True, alpha=0.3)

        # QPSK com ruído
        axes[2, 1].plot(tempo_qpsk, sinal_qpsk_ruidoso, 'r-', linewidth=1, alpha=0.7)
        axes[2, 1].set_title(f'Sinal QPSK com Ruído', fontsize=12, fontweight='bold')
        axes[2, 1].set_xlabel('Tempo (s)', fontsize=10)
        axes[2, 1].set_ylabel('Amplitude', fontsize=10)
        axes[2, 1].grid(True, alpha=0.3)

        plt.tight_layout()

        nome_arquivo = f'sinais_SNR_{snr_db:+03d}dB.png'
        plt.savefig(nome_arquivo, dpi=300, bbox_inches='tight')

        print(f"✓ Gráfico salvo: {nome_arquivo}")

    print(f"\n{len(list(snr_range))} gráficos de sinais foram gerados com sucesso!")

## Main

In [ ]:
if __name__ == "__main__":
    print("Configs")
    print("=" * 70)

    # 1) Texto
    mensagem_teste = input("Digite a mensagem a ser transmitida: ")

    # 2) Quantidade de amostras por bit
    while True:
        try:
            amostras_por_bit = int(input("Digite o número de amostras por bit (ex: 10, 50, 100): "))
            if amostras_por_bit <= 0:
                print("O número de amostras deve ser maior que zero.")
                continue
            break
        except ValueError:
            print("Valor inválido. Digite um número inteiro.")

    # 3) Faixa de SNR (em dB) para os gráficos
    while True:
        try:
            snr_min = int(input("Digite o valor mínimo de SNR em dB (ex: -5): "))
            snr_max = int(input("Digite o valor máximo de SNR em dB (ex: 15): "))
            if snr_max < snr_min:
                print("O valor máximo deve ser maior ou igual ao mínimo.")
                continue
            break
        except ValueError:
            print("Valor inválido. Digite números inteiros.")

    # Monta o range de SNR
    snr_range = range(snr_min, snr_max + 1)

    print("\n" + "=" * 70)
    print("ANÁLISE DE DESEMPENHO - BER vs SNR")

    # Gráfico comparativo:
    plotar_ber_vs_snr_comparativo(mensagem_teste, snr_range=snr_range, amostras_por_bit=amostras_por_bit)

    print("\n" + "=" * 70)
    print("GRÁFICOS DE ONDAS PARA CADA SNR")

    # Gráfico de Ondas
    plotar_sinais_exemplo(mensagem_teste, snr_range=snr_range, amostras_por_bit=amostras_por_bit)

    print("\n" + "=" * 70)
    print("Análise concluída!")
    print("=" * 70)